In [ ]:
!pip install pyro-ppl --quiet

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import numpy as np

# Enable deterministic behavior for reproducibility (optional)
pyro.set_rng_seed(42)


In [ ]:
# True function: h(x) = 1 + 3x
def true_head(x):
    return 1.0 + 3.0 * x

# Generate data
num_points = 20
X = torch.linspace(0, 1, num_points)
h_true = true_head(X)

# Add synthetic measurement noise
noise_std = 0.1
h_obs = h_true + noise_std * torch.randn(num_points)

# Reshape for neural network usage
X = X.unsqueeze(-1)       # shape [20, 1]
h_obs = h_obs.unsqueeze(-1)  # shape [20, 1]


In [ ]:
def bnn_forward(x, weights, biases):
    """
    Forward pass of a simple 2-layer network:
      hidden layer => ReLU => output layer
    weights, biases are dictionaries containing:
      - w1, b1 for layer1
      - w2, b2 for layer2
    x is shape [batch_size, input_dim]
    """
    # Hidden layer
    h = torch.relu(torch.mm(x, weights['w1']) + biases['b1'])
    # Output layer
    out = torch.mm(h, weights['w2']) + biases['b2']
    return out


In [ ]:
hidden_dim = 10  # number of neurons in hidden layer

def model(x, y_obs=None):
    # PRIOR: Define Normal(0,1) over weights & biases (mean=0, std=1)
    # Layer 1
    w1 = pyro.sample("w1", dist.Normal(torch.zeros(1, hidden_dim), torch.ones(1, hidden_dim)))
    b1 = pyro.sample("b1", dist.Normal(torch.zeros(hidden_dim), torch.ones(hidden_dim)))
    
    # Layer 2
    w2 = pyro.sample("w2", dist.Normal(torch.zeros(hidden_dim, 1), torch.ones(hidden_dim, 1)))
    b2 = pyro.sample("b2", dist.Normal(torch.zeros(1), torch.ones(1)))
    
    # Predictions using the forward pass
    weights = {'w1': w1, 'w2': w2}
    biases  = {'b1': b1, 'b2': b2}
    y_pred = bnn_forward(x, weights, biases)
    
    # LIKELIHOOD: We assume Gaussian noise around predicted y
    sigma = pyro.sample("sigma", dist.Exponential(torch.tensor(1.0)))  # noise scale

    # Condition on observed data if provided
    with pyro.plate("data_plate", x.shape[0]):
        pyro.sample("obs", dist.Normal(y_pred, sigma), obs=y_obs)
    
    return y_pred


In [ ]:
def guide(x, y_obs=None):
    # For each prior, define variational distribution parameters
    w1_loc = pyro.param("w1_loc", torch.randn(1, hidden_dim))
    w1_scale = pyro.param("w1_scale", torch.ones(1, hidden_dim), constraint=torch.distributions.constraints.positive)
    
    b1_loc = pyro.param("b1_loc", torch.randn(hidden_dim))
    b1_scale = pyro.param("b1_scale", torch.ones(hidden_dim), constraint=torch.distributions.constraints.positive)
    
    w2_loc = pyro.param("w2_loc", torch.randn(hidden_dim, 1))
    w2_scale = pyro.param("w2_scale", torch.ones(hidden_dim, 1), constraint=torch.distributions.constraints.positive)
    
    b2_loc = pyro.param("b2_loc", torch.randn(1))
    b2_scale = pyro.param("b2_scale", torch.ones(1), constraint=torch.distributions.constraints.positive)

    sigma_loc = pyro.param("sigma_loc", torch.tensor(1.0), constraint=torch.distributions.constraints.positive)

    # SAMPLE from these distributions
    pyro.sample("w1", dist.Normal(w1_loc, w1_scale))
    pyro.sample("b1", dist.Normal(b1_loc, b1_scale))
    pyro.sample("w2", dist.Normal(w2_loc, w2_scale))
    pyro.sample("b2", dist.Normal(b2_loc, b2_scale))
    pyro.sample("sigma", dist.Exponential(sigma_loc))


In [ ]:
# Reset Pyro parameters
pyro.clear_param_store()

# Define optimizer
optimizer = Adam({"lr": 0.01})

# Define our SVI object
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

num_iterations = 3000
losses = []

for step in range(num_iterations):
    loss = svi.step(X, h_obs)
    losses.append(loss)
    if step % 500 == 0:
        print(f"Step {step}, Loss = {loss:.2f}")

# Plot the convergence of ELBO loss
plt.figure()
plt.plot(losses)
plt.title("ELBO Loss during training")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()


In [ ]:
# Number of posterior samples
num_samples = 200

x_plot = torch.linspace(0, 1, 100).unsqueeze(-1)
predictions = []

for _ in range(num_samples):
    # sample from the guide
    w1_sample = dist.Normal(pyro.param("w1_loc"), pyro.param("w1_scale")).sample()
    b1_sample = dist.Normal(pyro.param("b1_loc"), pyro.param("b1_scale")).sample()
    w2_sample = dist.Normal(pyro.param("w2_loc"), pyro.param("w2_scale")).sample()
    b2_sample = dist.Normal(pyro.param("b2_loc"), pyro.param("b2_scale")).sample()
    sigma_sample = dist.Exponential(pyro.param("sigma_loc")).sample()

    weights_samp = {'w1': w1_sample, 'w2': w2_sample}
    biases_samp  = {'b1': b1_sample, 'b2': b2_sample}
    
    y_pred_samp = bnn_forward(x_plot, weights_samp, biases_samp).detach().squeeze()
    predictions.append(y_pred_samp.numpy())

predictions = np.array(predictions)  # shape [num_samples, 100]
mean_pred = np.mean(predictions, axis=0)
std_pred = np.std(predictions, axis=0)

# Plot results
plt.figure(figsize=(8,6))
plt.plot(X.squeeze(), h_obs.squeeze(), 'o', label="Observations")
plt.plot(x_plot.squeeze(), true_head(x_plot).squeeze(), 'k--', label="True function")

plt.plot(x_plot.squeeze(), mean_pred, 'r', label="BNN Mean Prediction")
plt.fill_between(x_plot.squeeze(),
                 mean_pred - 2*std_pred,
                 mean_pred + 2*std_pred,
                 alpha=0.2,
                 color="red",
                 label="±2 SD")

plt.title("Bayesian Neural Network on 1D Flow Data")
plt.xlabel("x")
plt.ylabel("Head h(x)")
plt.legend()
plt.show()
